In [2]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re


page =  1
BASE_URL = "https://www.deccanherald.com/"
country = "bangladesh"
initial_url = f"{BASE_URL}search/{page}?q={country}"


#NEED TO FIRST LOAD USING SELENIUM THEN YOU CAN USE driver.page_source FOR FURTHER SCRAPING 

#COUNTING NUMBER OF PAGES AVAILABLE FOR BANGLADESH

#SINCE IT'S A HUGE NUMBER, USING FIRST 2-3 PAGES FOR EXAMPLE. WHILE LOOP WORKS TO FIND TOTAL PAGES. 

count = 0

for page in range(1,4):
    
    initial_url = f"{BASE_URL}search/{page}?q={country}"

    time.sleep(5)

    driver = webdriver.Chrome()
    driver.get(initial_url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    #CONTENT DIV WILL BE AVAILABLE IF PAGE EXISTS.
    
    content_div_available = soup.find('div', class_ = 'listing-story-card hide-mobile Z6Zei')
    
    if not content_div_available:
        break
        
    else:
        page+=1
        count+=1
    
    time.sleep(5)
    
        
print(f'Number of pages available for Bangladesh is {count}')

#SCRAPING ALL NEWS LINKS FOR FIRST TWO PAGES
news_links = []

for page in range(1,3):
    
    initial_url = f"{BASE_URL}search/{page}?q={country}"
    
    time.sleep(5)
    
    driver = webdriver.Chrome()
    driver.get(initial_url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    main_div = soup.find_all('div', class_ = 'listing-story-card hide-mobile Z6Zei')

    for link_div in main_div:

        link = link_div.find('a').get('href')
        news_links.append(link)
    
    time.sleep(5)
        
    driver.quit()
    
print(f'Number of news links scraped: {len(news_links)}')


#SCRAPING THE NEWS SITES

data_list = []
counter = 0

for url in news_links:
    
        link = f'{BASE_URL}{url}'

        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')

        country = 'bangladesh'

        #TITLE 
        title_tag = soup.find('meta', {'name':'title'})
        title  = title_tag.get('content') if title_tag else 'Title not found'
        title_translation = 'None'

        #AUTHOR

        author_tag = soup.find('meta', {'name':'author'})
        author  = author_tag.get('content') if author_tag else 'Author not found'
        

        #DATE

        date_data = soup.find('div', class_ = 'ehN3D')

        if date_data:

            cleaned_date_data = date_data.text.split('Updated ')[1].split(' IST')[0]
            source_localtime = datetime.strptime(cleaned_date_data, '%d %B %Y, %H:%M')
            bangladesh_localtime = source_localtime + timedelta(minutes = 30)

        else:

            date_data = 'Date data not found'

        #CONTENT SUMMMARY 

        content_summary_tag = soup.find('div', {'data-test-id':'subheadline'})
        content_summary = content_summary_tag.text if content_summary_tag else 'Content Summary not found'
        summary_translation = 'None'

        

        #CONTENT

        content =[]

        all_content_divs = soup.find_all('div', {'id':'text-element-with-ad'})

        for each_content_div in all_content_divs:
            content.append(each_content_div.text)

        full_content = ''.join(content)
        full_content = re.sub('\xa0',' ',full_content)
        full_content = re.sub('\n|\r',' ',full_content)
        content_translation = 'None'
        
        
        data_dict = {
        "url": link,
        "title": title,
        "content": full_content,
        "content_summary": content_summary,
        "title_translation":title_translation,
        "content_translation":content_translation,
        "summary translation":summary_translation,
        "author": author,
        "country": country,
        'source_localtime': source_localtime,
        'bangladesh_localtime': bangladesh_localtime

    }

        counter+=1


        if (full_content != "Content Not Found" and content_summary != 'Content summary not found'):

                if data_dict not in data_list:
                        # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')
                
                
df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_Deccan_Herald.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="bangladesh_localtime", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


Number of pages available for Bangladesh is 3
Number of news links scraped: 20
Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
Link 12 added
Link 13 added
Link 14 added
Link 15 added
Link 16 added
Link 17 added
Link 18 added
Link 19 added
Link 20 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://www.deccanherald.com//india/mizoram/mi...,Mizoram not to collect biometric data of Myanm...,Aizawl: Mizoram Chief Minister Lalduhoma on Th...,The Chief Minister said that the Centre is cur...,None,None,None,PTI,bangladesh,2024-02-29 10:15:00,2024-02-29 10:45:00
1,https://www.deccanherald.com//india/india-bang...,"India, Bangladesh DG-level border talks in Dha...",New Delhi: India and Bangladesh will hold thei...,A delegation led by Border Security Force (BSF...,None,None,None,PTI,bangladesh,2024-02-29 06:48:00,2024-02-29 07:18:00
2,https://www.deccanherald.com//india/india-bang...,"India, Bangladesh recognise their constitution...",Dhaka: Chief Justice of India D Y Chandrachud ...,The CJI spoke at the valedictory function of a...,None,None,None,PTI,bangladesh,2024-02-24 16:16:00,2024-02-24 16:46:00
3,https://www.deccanherald.com//world/eight-rohi...,Eight Rohingya refugees injured in fire at cam...,Dhaka: Eight Rohingya refugees were injured on...,"Bangladesh has relocated around 32,000 people ...",None,None,None,Reuters,bangladesh,2024-02-24 10:11:00,2024-02-24 10:41:00
4,https://www.deccanherald.com//india/biz-govt-l...,"Government allows 54,760 tons of onion exports...","New Delhi: Amid export ban on onion, India on ...",Consumer Affairs Secretary Rohit Kumar Singh s...,None,None,None,PTI,bangladesh,2024-02-22 16:01:00,2024-02-22 16:31:00
